# Filings Monthly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [12]:
import os
import psycopg2
import pandas as pd
import matplotlib
from datetime import datetime, timedelta
from IPython.core.display import HTML

%load_ext sql
%config SqlMagic.displaylimit = 5

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


This will create the connection to the database and prep the jupyter magic for SQL

In [15]:
connect_to_db = 'postgresql://' + \
                os.getenv('PG_USER', '') + ":" + os.getenv('PG_PASSWORD', '') +'@' + \
                os.getenv('PG_HOST', '') + ':' + os.getenv('PG_PORT', '5432') + '/' + os.getenv('PG_DB_NAME', '');
    
%sql $connect_to_db

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

monthly total before running time.

In [ ]:
%%sql stat_filings_monthly_completed  <<
SELECT b.identifier AS COOPERATIVE_NUMBER
       , b.legal_name AS COOPERATIVE_NAME
       , COUNT(b.identifier) AS FILINGS_TOTAL_COMPLETED
       , STRING_AGG(f.filing_type, ', ')  AS FILING_TYPES_COMPLETED
FROM businesses b,
filings f
WHERE b.id = f.business_id
AND f.status='COMPLETED'
AND date(f.completion_date at time zone 'utc' at time zone 'pst')  > date(current_date - 1 - interval '1 months')
GROUP BY b.identifier, b.legal_name
UNION ALL
SELECT 'SUM' identifier, null, COUNT(b.identifier) AS count, null
FROM businesses b,
filings f
WHERE b.id = f.business_id
AND f.status='COMPLETED'
AND date(f.completion_date at time zone 'utc' at time zone 'pst')  > date(current_date - 1 - interval '1 months');

In [ ]:
edt = stat_filings_monthly_completed.DataFrame()

Save to CSV

In [ ]:
filename = 'monthly_total_till_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m-%d') +'.csv'
edt.to_csv(filename, sep=',', encoding='utf-8', index=False)